# Preparing to compute with TensorFlow

In [1]:
import Paths

# setting up the paths
working_paths = Paths.WorkingPaths()
working_paths.setup_paths()

Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images\collected_images
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images\collected_images_resized
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images\trainset
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images\testset
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\images\devset
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\scripts
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\models
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\annotations
Creating c:\dev\DHBW\Studienarbeit\Detection_training\Tensorflow\workspace\models
Creating c:\dev\DHBW\Studienarbeit\Detection_training

# Labeling own images with Label Studio

In [ ]:
import os

!pip install --upgrade pyqt5 lxml

if os.listdir(working_paths.LABELIMG_PATH).__len__ == 0:
    !git clone https://github.com/tzutalin/labelImg {working_paths.LABELIMG_PATH}

if os.name == 'posix':
    !cd {working_paths.LABELIMG_PATH} && make qt5py3
if os.name =='nt':
    !cd {working_paths.LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

Additionally run it

In [ ]:
!cd {working_paths.LABELIMG_PATH} && python labelImg.py

# Downloading images from other sources

First i downloaded the images and xml-annotations from this [github repository](https://github.com/datitran/raccoon_dataset).

You can simply copy them in the `collected_images` directory which should be available under the `Tensorflow/workspace/images` directory. Now you should have about 200 images from Racoons pretty low number isn't it?

Well thats true so we will increase them. But for a first evaluation of choosing the right pretrained model it should be fine to use. Especially if you want to use Google Colab since you must likely an only train for a couple of hours before you run out of GPU.

# Preprocessing the images for better performance during trainings.

downscaling thm now is better than when TensorFlow would do it on runtime

In [ ]:
from preprocessing_data import resize_images, preprocess_annotations

resize_images(working_paths.COLLECTED_IMAGES_PATH, working_paths.RESIZED_IMAGES_PATH, 320)
preprocess_annotations(working_paths.COLLECTED_IMAGES_PATH, working_paths.RESIZED_IMAGES_PATH, 320)

# Split up the resized images into Dev- Test- and Trainset

You can do it manually or using the following Code snippet to do it

In [ ]:
import os

# First delete existing files in the directory for the datasets
def delete_datasets(directory: str):
    for f in os.listdir(directory):
        os.remove(os.path.join(directory, f))

delete_datasets(working_paths.DEVSET_PATH)
delete_datasets(working_paths.TESTSET_PATH)
delete_datasets(working_paths.TRAINSET_PATH)

all_files: dict[list] = {}
# Getting all the files in the directory mapped to the basenames (img, xml-file)
for file in os.listdir(working_paths.RESIZED_IMAGES_PATH):
    filename = file.split('.', 1)[0]
    if filename in all_files:
        all_files[filename].append(file)
    else:
        all_files[filename] = [file]

# determine the ratio  for dev- and testset files
testset_files = 0.1
devset_files = 0.1
# rest will be used for training

position = 0
output_dir = working_paths.TESTSET_PATH
# lets start with testset files
for filename, names in all_files.items():
    if position >= testset_files * len(all_files):
        output_dir = working_paths.DEVSET_PATH
    if position >= (testset_files + devset_files) * len(all_files):
        output_dir = working_paths.TRAINSET_PATH

    for name in names:
        original_path = os.path.join(working_paths.RESIZED_IMAGES_PATH, name)
        output_path = os.path.join(output_dir, name)
        os.replace(original_path, output_path)
    position += 1


# Compress Datasets to use on Google Colab and on other platforms

In [4]:

import os
command = "{} {} {} {}".format(os.path.join(working_paths.IMAGE_PATH, 'dataset.tar.gz'), working_paths.TRAINSET_PATH, working_paths.TESTSET_PATH, working_paths.DEVSET_PATH)
!tar -czf {command}
# If you want to export the dataset you need to manually copy it

tar: Removing leading drive letter from member names
